In [1]:
import json
import requests
import pandas as pd
from pandas import json_normalize
import numpy as np
from bs4 import BeautifulSoup
from datetime import datetime
import os.path
from os import path
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
dfGames = pd.read_csv("games.csv")
df_train = pd.read_csv("games_playercount_train.csv")
df_test = pd.read_csv("games_playercount_test.csv")

bag_train = bag_test = pd.Series(dtype="float64")        #tried following the format from lecture notebooks but results weren't accurate when displaying so i have this for now
descDF_train = descDF_test = pd.DataFrame()
descArr_train = descArr_test = []

for i in range(len(df_train)):     
    appendGenre_train = "GENRE"
    appendGenre_train+=str(df_train['Genre'][i]).replace(" ", "")
    appendGenre_train+=" "
    df_train["Description"][i] = appendGenre_train + df_train["Description"][i]
    
    appendGenre_test = "GENRE"
    appendGenre_test+=str(df_test['Genre'][i]).replace(" ", "")
    appendGenre_test+=" "
    df_test["Description"][i] = appendGenre_test + df_test["Description"][i]

<ipython-input-2-277bfaaaa3cb>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train["Description"][i] = appendGenre_train + df_train["Description"][i]
<ipython-input-2-277bfaaaa3cb>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test["Description"][i] = appendGenre_test + df_test["Description"][i]


The for loop above allows us to add a "weight" to the game's previous genre on roblox.com. We will be using TF_IDF to determine to weights of each word in the description and this string concatenation to the game's description allows us to set a weight to it. 

In [3]:
for i in range(df_train.shape[0]):
    gameDesc = df_train["Description"][i]
    gameName = df_train["Name"][i]
    descSeries_train = pd.Series(gameDesc, index=[gameName])
    descArr_train.append(descSeries_train)
    bag_train = (
        descSeries_train.str.lower().str.replace("!", " ").str.split()
    ).apply(Counter)
    descDF_train = descDF_train.append(list(bag_train), ignore_index=True)
    descDF_train = descDF_train.fillna(0)

for j in range(df_test.shape[0]):
    gameDesc = df_test["Description"][j]
    gameName = df_test["Name"][j]
    descSeries_test = pd.Series(gameDesc, index=[gameName])
    descArr_test.append(descSeries_test)
    bag_test = (
        descSeries_test.str.lower().str.replace("!", " ").str.split()
    ).apply(Counter)
    descDF_test = descDF_test.append(list(bag_test), ignore_index=True)
    descDF_test = descDF_test.fillna(0)

Both for loops are essentially the same. They grab the corresponding game's description from the csv file and manipulates it so we have only alphabetical characters. 

In [4]:
docFreq_train = (descDF_train > 0).sum(axis=0)
idf_train = np.log(descDF_train.shape[0]/docFreq_train)
tf_idf_train = descDF_train * idf_train

tfidf_train_List = list(tf_idf_train)
for i in range(len(tfidf_train_List)):
    if not tfidf_train_List[i].isalpha():
        tf_idf_train = tf_idf_train.drop([tfidf_train_List[i]], axis=1)    #if column name(word in game description) does not contain character from alphabet, drop entire column
tf_idf_train

,genreadventure,welcome,to,anime,fighters,code,at,likes,thanks,for,...,appliances,cooks,waiters,foods,exchange,global,marketplace,rich,programmer,david
0,1.560648,1.386294,0.931648,2.995732,18.420681,2.525729,1.560648,4.240527,2.65926,1.156987,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
1,0.000000,0.000000,0.745318,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,1.156987,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
2,0.000000,0.000000,0.372659,0.000000,0.000000,0.000000,0.000000,0.000000,2.65926,0.385662,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
3,0.000000,0.000000,0.558989,0.000000,0.000000,5.051457,1.560648,2.120264,0.00000,1.542650,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
4,0.000000,0.000000,0.372659,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.771325,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.000000,0.000000,0.186330,0.000000,0.000000,0.000000,0.000000,2.120264,0.00000,1.542650,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
96,1.560648,1.386294,1.117977,0.000000,0.000000,0.000000,1.560648,0.000000,0.00000,0.000000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
97,1.560648,1.386294,0.372659,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.385662,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
98,0.000000,1.386294,0.558989,2.995732,0.000000,0.000000,0.000000,0.000000,0.00000,0.771325,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000


In [19]:
docFreq_test = (descDF_test > 0).sum(axis=0)
idf_test = np.log(descDF_test.shape[0]/docFreq_test)
tf_idf_test = descDF_test * idf_test

tfidf_test_List = list(tf_idf_test)
for i in range(len(tfidf_test_List)):
    if not tfidf_test_List[i].isalpha():
        tf_idf_test = tf_idf_test.drop([tfidf_test_List[i]], axis=1)    #if column name(word in game description) does not contain character from alphabet, drop entire column

tf_idf_test

,genrefighting,welcome,to,dragon,blox,previously,known,as,super,saiyan,...,efficiently,larger,nozzles,miss,sub,fluff,bluegrass,starmaq,steam,suggestions
0,1.609438,1.237874,0.466135,4.60517,3.912023,3.912023,3.506558,1.469676,2.995732,4.60517,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
1,0.000000,0.000000,0.233068,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
2,0.000000,1.237874,0.116534,0.00000,0.000000,0.000000,0.000000,1.469676,0.000000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
3,0.000000,0.000000,0.582669,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
4,0.000000,0.000000,0.349601,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.000000,1.237874,0.116534,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
96,0.000000,1.237874,0.932271,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
97,0.000000,1.237874,0.116534,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
98,0.000000,0.000000,0.116534,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000


Both tables above show the tf_idf dataframe which applies the log function to account for each term that appears in each game's description. 

In [20]:
df_test["NewGenre"] = ""
for i in range(tf_idf_test.shape[0]):
    a = tf_idf_test.iloc[i]
    b = tf_idf_train
    dot = a.multiply(b).sum(axis=1)
    
    a_len = np.sqrt(a.multiply(a).sum())
    b_len = np.sqrt(b.multiply(b).sum(axis=1))
    
    dot / (a_len * b_len)

    cos_similarities = pd.DataFrame(dot / (a_len * b_len))[0]
    most_similar = cos_similarities.sort_values(ascending=False)
    most_similar
    
    mostSimilarIndex = most_similar.index[0]
    df_test.at[i, "NewGenre"] = df_train.at[mostSimilarIndex, 'NewGenre']
    print(df_test['GameDetailReferralUrl'][i], ": ", df_test['NewGenre'][i])

https://www.roblox.com/games/3177438863/Update-6-Dragon-Blox :  Anime
https://www.roblox.com/games/3233893879/NEW-GUN-Bad-Business-2-45 :  Action
https://www.roblox.com/games/5780309044/Stands-Awakening-Beta :  Anime
https://www.roblox.com/games/4543144283/GUESTS-Mega-Noob-Simulator :  Simulator/Tycoon
https://www.roblox.com/games/155382109/Survive-and-Kill-the-Killers-in-Area-51 :  Roleplay
https://www.roblox.com/games/4110892437/MGII-Kaiju-Universe :  Action
https://www.roblox.com/games/2512643572/SUPER-LUCK-Bubble-Gum-Simulator :  Simulator/Tycoon
https://www.roblox.com/games/5670292785/TANK-Undead-Defense-Tycoon :  Building
https://www.roblox.com/games/2569453732/Updated-Peace-Treaties-Rise-of-Nations :  Anime
https://www.roblox.com/games/3351674303/Driving-Empire :  Simulator/Tycoon
https://www.roblox.com/games/155615604/Prison-Life-Cars-fixed :  Simulator/Tycoon
https://www.roblox.com/games/5712833750/Animal-Simulator :  Anime
https://www.roblox.com/games/443406476/Project-Lazaru

From here we can apply cosine similarity to compare our testing dataset with the training dataset. We will then output the game's website link along with the new genre that our model has predicted for it. 

In [22]:
df_test.to_csv(path_or_buf="games_playercount_newgenres.csv", index=False)